<a href="https://colab.research.google.com/github/Elsacred02/assignment_4_advanced_machine_learning/blob/main/Assignment4_parte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras import layers

keras.utils.set_random_seed(42)

In [ ]:
path = 'divina_commedia.txt'
with open(path, encoding='utf-8') as f:
  text = f.read().lower()
print('Text length:', len(text))
print()
print(text[0:1000])

FileNotFoundError: [Errno 2] No such file or directory: 'divina_commedia.txt'

In [ ]:
chars = sorted(list(set(text)))

print('total chars:', len(chars))

#from chars to numbers
char_indices = dict((c, i) for i, c in enumerate(chars))
# from numbert to chars
indices_char = dict((i, c) for i, c in enumerate(chars))

print(char_indices)
print(indices_char)



In [ ]:
maxlen = 30
step = 2

sentences = []
next_chars = []

for i in range(0, 200000, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])

print('number of sentences:', len(sentences))

print(sentences[11])
print(next_chars[11])

In [ ]:
x = np.zeros((len(sentences), maxlen, len(chars))) # 100,000 x 30 x 40
y = np.zeros((len(sentences), len(chars))) # 100,000 x 40

for i, (sentence, next_char) in enumerate(zip(sentences, next_chars)):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next_char]] = 1

print(x.shape)
print(y.shape)

In [ ]:
model = keras.Sequential([
    keras.Input(shape=(30, 40, 1)),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(len(chars), activation="softmax"),
])

optimizer = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

In [ ]:
from keras.callbacks import LambdaCallback
import sys

def testAfterEpoch(epoch, _):
  if epoch < 10:
    return
  print()
  print()
  print('----- Generating text after Epoch: %d' % epoch)

  start_index = random.randint(0, len(text)-maxlen-1)

  generated = ''
  sentence = text[start_index: start_index + maxlen]
  generated += sentence
  print('**** starting sentence*****')
  print(sentence)
  print('***************************')
  sys.stdout.write(generated)

  for i in range(200):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
      x_pred[0, t, char_indices[char]] = 1
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = np.argmax(preds)
    next_char = indices_char[next_index]

    sentence = sentence[1:] + next_char
    sys.stdout.write(next_char)
    sys.stdout.flush()
  print()

print_callback = LambdaCallback(on_epoch_end=testAfterEpoch)



In [ ]:
model.fit(x,y,
          batch_size=2048,
          epochs=30,
          callbacks = [print_callback])